# Coding Best Practices

When first getting started with coding, developers often write code that is "good enough" and then stop making improvements. However, this code may be hard to understand, contain bugs, and be hard to reuse or extend. Professional software developers have developed best practices to help avoid these problems.

## Code style

Python is mostly pretty flexible about how code can be formatted. But there is a standard code style that is easy to use and helps make your code easier to read.

Here is some code before formatting. With the random use of spaces and long lines of code, this is pretty hard to read.

In [1]:
import polars as pl
from datascipsych import datasets

def myfunction( x, y ):
    #add some numbers
    z  = x+y
    return z
l=[1,2,3,4]
d={'a':1,"b":2,"c":3}
df = pl.read_csv(datasets.get_dataset_file("Morton2013"), null_values="n/a").filter(pl.col("study")).group_by("subject", "list_type", "input").agg(pl.col("recall").mean())

Luckily, we can use [Black](https://black.readthedocs.io/en/stable/), a tool for automatic reformatting of Python code, to reformat it. Black has different ways of running it, including a command line tool and a plugin for VSCode. Now the code is much easier to read.

In [2]:
import polars as pl
from datascipsych import datasets


def myfunction(x, y):
    # add some numbers
    z = x + y
    return z


l = [1, 2, 3, 4]
d = {"a": 1, "b": 2, "c": 3}
df = (
    pl.read_csv(datasets.get_dataset_file("Morton2013"), null_values="n/a")
    .filter(pl.col("study"))
    .group_by("subject", "list_type", "input")
    .agg(pl.col("recall").mean())
)

Black automatically reformats to match Python formatting guidelines, plus some additional rules that Black uses to increase consistency. The name "Black" comes from a quote from Henry Ford about the Model T car: "Any customer can have a car painted any color that he wants so long as it is black".

I used to have a lot of recommendations for how to format code. Now I tell people: "just use Black."

Of course, Black won't change anything about how the code runs, so there are some recommended guidelines that it won't implement. For example, it's recommended that module import statements be placed at the top of a module. This makes it easier to see what modules are being used in the file and how they are named.

In [3]:
def myfunction(x, y):
    # add some numbers
    z = x + y
    return z


import numpy as np  # not recommended (comes after other code)
a = np.arange(6)

We should generally move the import statement to the top of the file before other code, unless there's a good reason to import it somewhere else.

In [4]:
import numpy as np


def myfunction(x, y):
    # add some numbers
    z = x + y
    return z


a = np.arange(6)

Note also that Python style guidelines recommend having two lines above and below each function definition, to make them easier to spot separately from other code.

### Exercise: code style

Use Black to reformat the following code. Also, make it so the import statements are all at the top of the cell.

In [5]:
import numpy as np
b = np.zeros((1,2))
import polars as pl
data = pl.DataFrame({"trial":[1,2,3,4], "correct":[0,1,1,0], "response_time":[1.2,3.4,2.3,5.6]})

## Coding principles

Code developers have tried to explain the difference between good and bad code in various ways. We'll review a few useful principles that can help guide your programming.

### The Zen of Python

The [Zen of Python](https://peps.python.org/pep-0020/) is a set of principles written by an early Python developer to describe how to write good Python code.

Here are a few of the core principles:

Beautiful is better than ugly. (Good code is elegant.)

Explicit is better than implicit. (Communicate your intentions.)

Simple is better than complex. (Simplify when possible.)

Readability counts. (Code is read more often than it is written, so make it easy to read.)

There should be one-- and preferably only one --obvious way to do it. (Python tries to make it obvious how to do something.)